In [1]:
from imap_functions import *
import matplotlib.pyplot as plt


##### EXAMPLE RETRIEVAL FOR SCENE OVER SAN JUAN POWER PLANT


#Load data
class args:
    pass

args.direc = 'prs20200731t181747_SanJuanMeth'
args.name = 'prs20200731t181747_SanJuanMeth'
args.lat = 36.793
args.lon = 108.391
args.source = 'SanJuanMeth'

#Using meteorological fields and HITRAN cross-sections precomputed from a different
#scene. They should work fine for illustration
args.met = 'prs20191115t171641_Blowout'
args.hitran = 'prs20191115t171641_Blowout'


#Set up the retrieval
init_dict = {'rad_dir': args.direc,\
             'rad_name': args.name,\
             'main': '/Users/cusworth/Documents/PRISMA/FullScenes/SanJuanMeth/',\
             'wave_pos': 'ancillary/prisma_wvl.txt',\
             'alt_agl_km': 500,\
             'latitude': float(args.lat),\
             'longitude': -1*float(args.lon),\
             'fwhm': 10,\
             'SNR': 100,\
             'deg_poly': 4,\
             'do_legendre': False, \
             'met_file':'met/merra_met'+ args.met + '.p',\
             'inversion_wvl': [2250, 2415], \
             'hitran_loaded': args.hitran}


#Load image and data
files, set_inputs, wave_class, set_flight, coord_sub = initialize(init_dict)
met = load_met(set_inputs, set_flight, files)
cs, met, wave_class = load_hitran_solar(wave_class, met, files)
rgb, rad_sub, wave_class, met = load_raw_observations(files, set_inputs, wave_class, met, cs)


HAPI version: 1.1.0.9.4
To get the most up-to-date version please check http://hitran.org/hapi
ATTENTION: Python versions of partition sums from TIPS-2017 are now available in HAPI code

           It is free to use HAPI. If you use HAPI in your research or software development,
           please cite it using the following reference:
           R.V. Kochanov, I.E. Gordon, L.S. Rothman, P. Wcislo, C. Hill, J.S. Wilzewski,
           HITRAN Application Programming Interface (HAPI): A comprehensive approach
           to working with spectroscopic data, J. Quant. Spectrosc. Radiat. Transfer 177, 15-30 (2016)
           DOI: 10.1016/j.jqsrt.2016.03.005


/Users/cusworth/Documents/IMAP_DOAS/imap_functions.py:553: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "load_raw_observations" failed type inference due to: NameError: name 'coord_sub' is not defined
  @jit
/Users/cusworth/opt/anaconda3/envs/py3/lib/python3.7/site-packages/numba/core/object_mode_passes.py:152: NumbaWarning: Function "load_raw_observations" was compiled in object mode without forceobj=True.

File "imap_functions.py", line 554:
@jit
def load_raw_observations(files, set_inputs, wave_class, met, cs):
^

  state.func_ir.loc))
/Users/cusworth/opt/anaconda3/envs/py3/lib/python3.7/site-packages/numba/core/object_mode_passes.py:162: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-object-mode-fall-back-behav

FileNotFoundError: Unable to locate file "/Users/cusworth/Documents/PRISMA/FullScenes/SanJuanMeth//prs20200731t181747_SanJuanMeth/prs20200731t181747_SanJuanMeth.hdr". If the file exists, use its full path or place its directory in the SPECTRAL_DATA environment variable.

In [ ]:
### Retrieval equations - in the more operational code, this is called "run_retrieval2"

#~~~~ INDICES IN DATA CUBE TO PLAY AROUND WITH ~~~~~
iindex = 50
jindex = 60

#Run retrieval given indices
Y = rad_sub[iindex,jindex,:] / .1
Y = Y[wave_class.wvl_sel]

inv_prms = make_class()

#Prior start of polynomial fit is fit to subsample of Y
init_k = set_inputs.deg_poly
YT = Y/met.T_lo_0

hi_x = np.linspace(-1, 1, len(wave_class.wvl_hi))
lo_x = interpolate.interp1d(np.flip(wave_class.wvl_hi), hi_x)(wave_class.wvl_lo)

#Fit the polynomial
near_zeros = np.where(wave_class.cs_low < np.percentile(wave_class.cs_low, 40))[0]
lfit = np.polynomial.legendre.legfit(lo_x[near_zeros], YT[near_zeros], init_k)
lval = np.polynomial.legendre.legval(hi_x, lfit)
lval_lo = np.polynomial.legendre.legval(lo_x, lfit)

#Scaling factors for gases
lCH4 = set_inputs.layers_ch4
lH2O = set_inputs.layers_h2o
lN2O = set_inputs.layers_n2o

#Define first prior
s1 = get_scaling(met.vmr_CH4, met.ch4_red_A, met.bnd_ch4, met.pmid, met.h_full)
s2 = get_scaling(met.vmr_H2O, met.h2o_red_A, met.bnd_h2o, met.pmid, met.h_full)
s3 = get_scaling(met.vmr_N2O, met.n2o_red_A, met.bnd_n2o, met.pmid, met.h_full)

#Set value of first prior + first iteration
xa_full = np.append(s1, np.append(s2, np.append(s3, np.append(wave_class.shift_coef, lfit))))
inv_prms.xa_full = xa_full

#Construct prior error covariance
sig_prior = np.abs(xa_full)  #Error on prior proportional to magnitude

#Methane prior
sig_prior[0] *= 1
if set_inputs.layers_ch4 == 2: #If airbone
    sig_prior[1] *= 1    
sig_prior[(lCH4):(lCH4+lH2O)] *= .5 #H2O prior
sig_prior[(lCH4+lH2O):(lCH4+lH2O+lN2O)] *= 1e-1 #N2O prior
sig_prior[lCH4+lH2O+lN2O] *= 1e-5 #Shift prior
sig_prior[-len(lfit):] *= 1e-2 #polynomial
Sa = np.diag((sig_prior**2))
invSa = np.linalg.inv(Sa)

#Obs error covariance
noise_val = 1/(set_inputs.SNR**2)
invSe = np.diag([1/noise_val] * len(Y))

#Do five iterations - generally this is sufficient
for i_iter in range(5):
    
    if i_iter == 0:
        ix = xa_full
    else:
        ix = x_1

    #Run Forward model
    ilval = np.polynomial.polynomial.polyval(hi_x, ix[-len(lfit):])
    T = np.flip(Transmission(ix[0:(lCH4)], ix[(lCH4):(lCH4+lH2O)], ix[(lCH4+lH2O):(lCH4+lH2O+lN2O)], met, cs))
    Fa = F_lo(Forward3(T, ilval), np.flip(wave_class.wvl_hi), wave_class.wvl_lo, set_inputs.fwhm)
    if i_iter==0:
        Fa_1 = Fa
    
    #Create Jacobian
    K = Make_Jac4(T, lcoefs=ix[-len(lfit):], scoefs=np.array([ix[lCH4+lH2O+lN2O]]), \
            wave_class=wave_class, met=met, set_inputs=set_inputs, cs=cs, \
            sCH4=ix[0:(lCH4)], sH2O=ix[(lCH4):(lCH4+lH2O)], sN2O=ix[(lCH4+lH2O):(lCH4+lH2O+lN2O)])

    #Iterative solution for next state
    term1 = np.linalg.inv(K.T.dot(invSe).dot(K) + invSa)
    term2 = K.T.dot(invSe)
    term3 = (Y - Fa) + K.dot(ix - xa_full)
    x_1 = xa_full + (term1.dot(term2)).dot(term3)

    #Posterior error
    S_1 = np.linalg.inv(K.T.dot(invSe).dot(K)+np.linalg.inv(Sa))
    lval_hat = np.polynomial.polynomial.polyval(hi_x, x_1[-len(lfit):])

#Posterior estimate
sCH4_hat = x_1[0:lCH4]
sH2O_hat = x_1[(lCH4):(lCH4+lH2O)]
sN2O_hat = x_1[(lCH4+lH2O):(lCH4+lH2O+lN2O)]

That = np.flip(Transmission(sCH4_hat, sH2O_hat, sN2O_hat, met, cs))
Fhat = F_lo(Forward3(That, lval_hat), np.flip(wave_class.wvl_hi), wave_class.wvl_lo, set_inputs.fwhm)

#Output data
inv_prms.Fh = Fhat
inv_prms.Y = Y
inv_prms.Fa = Fa_1
inv_prms.RMSE = np.sqrt(np.mean((Y-Fhat)**2))
inv_prms.ilval = lval_lo
inv_prms.lval_hat = F_lo(lval_hat, np.flip(wave_class.wvl_hi), wave_class.wvl_lo, set_inputs.fwhm)
inv_prms.K = K
inv_prms.xhat = x_1
inv_prms.Sa = Sa
inv_prms.Sh = S_1
inv_prms.Aker = np.eye(Sa.shape[0]) - S_1.dot(np.linalg.inv(Sa))

retrieved_CO2 = (np.dot(x_1[0:lCH4] * met.ch4_red_A, met.h) * 1e9) / 1000


In [ ]:
#Show output

print('Retrieval precision (%)', 100*np.sqrt(S_1[0,0]))
print('Prior precision (%)', 100*np.sqrt(Sa[0,0]))
print('Total XCO2 (ppb)', 1000*retrieved_CO2)

In [ ]:
#Plot averaging kernel for CO2
plt.plot(inv_prms.Aker[:,0], 'o-')
plt.title('Averaging kernel for XCO2')
plt.show()

In [ ]:
#Plot observed, prior, and posterior spectra

plt.plot(Y, label='Y')
plt.plot(Fa_1, label='Fa')
plt.plot(Fhat, label='Fh')
plt.legend()
plt.show()
